In [2]:
import pandas as pd
import numpy as np

from functions.cargar_data import *

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
atr1996 = pd.read_csv('tablas/atractividad_1996.csv',
                      index_col='deptos')
atr1996

In [ ]:
# guarda como numpy array 
atr1996_ar = np.array(atr1996)

In [ ]:
p_migr = cargar_migrantes_internos()
p_migr

In [ ]:
pobl = recuperar_poblacion_2011()
pobl.head(3)

In [ ]:
# genera un sólo dataframe solo para deptos
flujos_deptos = p_migr.loc[p_migr.depto_origen != p_migr.depto_destino, ['depto_origen', 'depto_destino']]
grupo = flujos_deptos.groupby(by=['depto_origen', 'depto_destino']).size()
grupo = pd.DataFrame(grupo, columns=['Mij']).reset_index()
grupo.head()

In [ ]:
# genera un sólo dataframe solo para deptos
flujos_deptos = p_migr.loc[p_migr.depto_origen != p_migr.depto_destino, ['depto_origen', 'depto_destino']]
Mji = list(flujos_deptos.groupby(by=['depto_destino', 'depto_origen']).size())
grupo['Mji'] = Mji
grupo.head()

In [ ]:
grupo_pob = grupo.merge(pobl, left_on='depto_origen', right_on='DPTO')
grupo_pob = grupo_pob.merge(pobl, left_on='depto_destino', right_on='DPTO')
grupo_pob.drop(['DPTO_x', 'DPTO_y'], axis=1, inplace=True)
grupo_pob.rename({'poblacion_x': 'Pi', 'poblacion_y': 'Pj'}, inplace=True, axis=1)
grupo_pob.head()

In [ ]:
# calcula cam
grupo_pob['cam'] = (grupo_pob.Mij + grupo_pob.Mji) / (grupo_pob.Pi + grupo_pob.Pj) * 1000

# redondea
grupo_pob['cam_round'] = round(grupo_pob.cam).astype(int)

In [ ]:
atr2011 = pd.pivot_table(grupo_pob,
                        values='cam_round',
                        index ='depto_origen',
                        columns='depto_destino',
                        fill_value=0).iloc[1:19, 0:18]

atr2011

In [ ]:
# convierte en array y reemplaza el triángulo superior por ceros
atr2011_ar = np.tril(np.array(atr2011))
print(atr2011_ar)

In [ ]:
np.nanmax((atr2011_ar - atr1996_ar))